In [22]:
from bs4 import BeautifulSoup
import re
import requests
import os

def apex():
    urls = [ f'https://apex-magazine.com/category/short-fiction/page/{i}' for i in range(1,11) ]
    soups = [BeautifulSoup(requests.get(url).text) for url in urls]
    stories = []
    for soup in soups:
        for article in soup.findAll('article'):
            date = article.find('span', attrs={'class':'elementor-post-date'}).text
            author = article.find('span', attrs={'class':'elementor-post-author'}).text
            title = article.find('h3').find('a').text
            href = article.find('h3').find('a').get('href')
            if '2022' in date:
                stories.append((title, author, href))
        # print(title, date)
    return stories

def bcs():
    url = 'https://www.beneath-ceaseless-skies.com/issues/2022/'
    soup = BeautifulSoup(requests.get(url).text)
    stories = []
    for issue in soup.findAll('div', attrs={'class':'post-body'}):
        for title in issue.findAll('div', attrs={'class':'issue-story-title'}):
            if title.previous_sibling.get('class') == ['issue-podcast-subheader']:
                continue
            author = title.next_sibling
            href = title.find('a').get('href')
            stories.append((title.find('a').text, author.text, href))

    return stories

def clarkesworld():
    url = 'https://clarkesworldmagazine.com/prior/?y=2022'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    stories = set()
    for table in soup.findAll('div', attrs={'class':'index-table'}):
        # if table doesn't contain a p tag with the words fiction, keep going
        section = table.find('p', attrs={'class': 'section'})
        if section is None or section.text.strip().lower() != 'fiction':
            continue
        for storydiv in table.findAll('div'):
    #         print(storydiv.text)
            title = storydiv.find('p', attrs={'class':'story'})
            author = storydiv.find('p', attrs={'class':'byline'})
            if not title or not author:
                continue
            href = title.find('a').get('href')
            title = title.text.strip()
            author = author.text.replace('by', '').strip()
            if "Editor's Desk" in title:
                continue
            if "A Conversation with" in title:
                continue
            if "An Interview with" in title:
                continue
            stories.add((title, author, href))
    return list(stories)

def metamorphosis():
    urls = [
        'https://magazine.metaphorosis.com/toc/2022/december-2022/',
        'https://magazine.metaphorosis.com/toc/2022/november-2022/',
        'https://magazine.metaphorosis.com/toc/2022/october-2022/',
        'https://magazine.metaphorosis.com/toc/2022/september-2022/',
        'https://magazine.metaphorosis.com/toc/2022/august-2022/',
        'https://magazine.metaphorosis.com/toc/2022/july-2022/',
        'https://magazine.metaphorosis.com/toc/2022/june-2022/',
        'https://magazine.metaphorosis.com/toc/2022/may-2022/',
        'https://magazine.metaphorosis.com/toc/2022/april-2022/',
        'https://magazine.metaphorosis.com/toc/2022/march-2022/',
        'https://magazine.metaphorosis.com/toc/2022/february-2022/',
        'https://magazine.metaphorosis.com/toc/2022/january-2022/'
    ]
    soups = [BeautifulSoup(requests.get(url).text) for url in urls]
    stories = []
    for soup in soups:
        for article in soup.find('div', {'class': 'entry-content'}).findAll('li', {'class': None}):
            title, author = article.text.split('—')
            href = article.find('a').get('href')
            stories.append((title.strip(), author.strip(), href))
    return stories


def nature():
    url = 'https://www.nature.com/nature/articles?type=futures&year=2022'
    soup = BeautifulSoup(requests.get(url).text)
    stories = []
    for article in soup.findAll('article'):
        title = article.find('h3')
        author = article.find('span').text
        href = title.find('a').get('href')
        stories.append((title.text.strip(), author, 'https://nature.com'+href))
    return stories

def strange_horizons():
    url = 'http://strangehorizons.com/issue/2022/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    stories = set()
    for article in soup.findAll('div', attrs={'class':'article'}):
        title = article.find('div', attrs={'class':'title'})
        try:
            href = title.find('a').get('href')
        except:
            print(f'{title.text} has no href, skipping')
            continue
        author = article.find('div', attrs={'class':'author'}).text.replace('By:', '').strip()
        if re.match(r"http.+/fiction/.+/$", href):
            stories.add((title.text.strip(), author, href))
    return stories

def tor():
    with open('All Fiction Tor.com pg1 2023-3-8.html') as f:
        soup1 = BeautifulSoup(f.read())
    with open('All Fiction Tor.com pg2 2023-3-8.html') as f:
        soup2 = BeautifulSoup(f.read())    
    with open('All Fiction Tor.com pg3 2023-3-8.html') as f:
        soup3 = BeautifulSoup(f.read())
    stories = set()
    for article in soup1.findAll('article') + soup2.findAll('article') + soup3.findAll('article'):
        # if table doesn't contain a p tag with the words fiction, keep going
        title = article.find('h2')
        date = article.find('div', attrs={'class':'entry-date'}).text
        author = article.find('a', attrs={'class':'entry-author'}).text
        href = title.find('a').get('href')
        if '2022' in date:
    #         print(title.text,date)
            stories.add((title.text.strip(), author, href))
    return stories

def uncanny():
    urls = [
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-four/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-five/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-six/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-seven/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-eight/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-nine/',
    ]
    soups = [BeautifulSoup(requests.get(url).text) for url in urls]
    stories = []
    for soup in soups:
        for p in soup.find('div', attrs={'class':'more_fiction'}).findAll('p'):
            try:
                title, author = p.findAll('a')
                author = author.text
            except:
                title, author, translator = p.findAll('a')
                author = f'{author.text} (translated by {translator.text})'
            stories.append((title.text, author, title.get('href')))
    return stories

In [23]:
mm_stories = metamorphosis()

<li><a href="https://magazine.metaphorosis.com/story/2022/the-dragons-due-christopher-warden/">The Dragon’s Due</a> — Christopher Warden</li>
<li><a href="https://magazine.metaphorosis.com/story/2022/through-the-middle-j-b-kish/">Through the Middle</a> — J.B. Kish</li>
<li><a href="https://magazine.metaphorosis.com/story/2022/love-death-and-the-electric-soul-alexandra-peel/">Love, Death, and the Electric Soul</a> — Alexandra Peel</li>
<li><a href="https://magazine.metaphorosis.com/story/2022/snowman-han-whiteoak/">Snowman</a> — Han Whiteoak</li>
<li><a href="https://magazine.metaphorosis.com/story/2022/infinite-possibilities-iv-michael-gardner/">Infinite Possibilities IV</a> — Michael Gardner</li>


In [2]:
apex_stories = apex()

In [3]:
bcs_stories = bcs()

In [4]:
cw_stories = clarkesworld()

In [5]:
na_stories = nature()

In [6]:
sh_stories = strange_horizons()

Podcast: 6 June Poetry  has no href, skipping
Podcast: 9 May Poetry  has no href, skipping
The Author and the Critic III: Alison Rumfitt and Marisa Mercurio  has no href, skipping
Podcast: Broken Blue  has no href, skipping


In [11]:
tor_stories = tor()

In [7]:
un_stories = uncanny()

In [8]:
def write_list(fname, stories):
    with open(fname, 'w') as f:
        for title, author, href in sorted(list(stories), key=lambda el: el[0]):
            f.write(f" - [{title}]({href}) by {author}\n")

In [15]:
for name,stories in {
    'apex': apex_stories, 
    'beneath-ceaseless-skies': bcs_stories,
    'clarkesworld': cw_stories,
    'nature-futures': na_stories,
    'strange-horizons': sh_stories,
    'tor': tor_stories,
    'uncanny': un_stories
}.items():
    write_list(name+'.md', stories)

In [14]:
output = ''
fname_match = '2022-shorts'
for fname in sorted(list(os.listdir('.'))):
    if fname[-3:] == '.md' and fname_match in fname:
        publication = fname.replace(fname_match, '').replace('.md', '').replace('-', ' ').title()
        output += f"\n## {publication} \n"
        with open(fname) as f:
            output += f.read() + "\n"

with open(fname_match+'.md', 'w') as f:
    f.write(output)
print("wrote to", fname_match+'.md')

wrote to 2022-shorts.md
